## Part 1) RDDs

In [1]:
// Intitialize Scala interpreter
sc

Intitializing Scala interpreter ...

Spark Web UI available at http://captain01.os.hpc.tuwien.ac.at:9999/proxy/application_1715326141961_1979
SparkContext available as 'sc' (version = 3.2.3, master = yarn, app id = application_1715326141961_1979)
SparkSession available as 'spark'


res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@53422aa9


In [2]:
// Load devset from HDFS
val reviews = spark.read.json("hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json")

// Load stopwords
val stopwordsPath = "stopwords.txt"
val stopwords = sc.textFile(stopwordsPath).collect()

reviews: org.apache.spark.sql.DataFrame = [asin: string, category: string ... 8 more fields]
stopwordsPath: String = stopwords.txt
stopwords: Array[String] = Array(a, aa, able, about, above, absorbs, accord, according, accordingly, across, actually, after, afterwards, again, against, ain, album, album, all, allow, allows, almost, alone, along, already, also, although, always, am, among, amongst, an, and, another, any, anybody, anyhow, anyone, anything, anyway, anyways, anywhere, apart, app, appear, appreciate, appropriate, are, aren, around, as, aside, ask, asking, associated, at, available, away, awfully, b, baby, bb, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, believe, below, beside, besides, best, better, between, beyond, bibs, bike, book,...


In [3]:
import scala.util.matching.Regex

// Define the regex pattern for splitting
val pattern: Regex = """[\s\t\d\(\)\[\]\{\}\.!\?,;:+=\-_"\'`~#@&\*%€$§\\/]+""".r

// Remove unnecessary data
val reviews_filtered = reviews.rdd.map{line =>
    val category = line.getAs[String] ("category")
    val text = line.getAs[String] ("reviewText")
    (category,text)}

// Split reviews, casefolding, remove stopwords, prepare key value pairs
val tokens = reviews_filtered.flatMap { case (category, text) =>
        pattern.split(text.toLowerCase)
        .filterNot(word => stopwords.contains(word))
        .map(word => ((category, word), 1))
}.cache()

import scala.util.matching.Regex
pattern: scala.util.matching.Regex = [\s\t\d\(\)\[\]\{\}\.!\?,;:+=\-_"\'`~#@&\*%€$§\\/]+
reviews_filtered: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[11] at map at <console>:31
tokens: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[12] at flatMap at <console>:37


In [4]:
// Count total number of reviews (N)
val N = reviews_filtered.count()

// Calculate value A and B
// ----> A value: all occurrences of unique (('Category', 'value'), 1) pairs
// ----> B value: every occurrence of a word across all categories, reduced by value A 
val A = tokens.reduceByKey(_ + _) // A value
val wordSum = A.map { case ((category, word), sum) => (word, sum) }.reduceByKey(_ + _) // Occurrence of words across all categories
val AB = A.keyBy { case ((category, word), sum) => word }
              .join(wordSum) // Join A and allCat and calculate B: ((Category, Word), A, B)
              .map { case (word, (((category, word2), sum), totalSum)) => ((category, word2), sum, totalSum - sum) }

// ----> C value
// Calculate total number of review per category
// And reduce it by all occurrences of the word in the current category (A)
val allCat = reviews_filtered.map { case (category, review) => (category, 1) }
                                          .reduceByKey(_ + _)

val ABC = AB.map { case ((category, term), valA, valB) => (category, (term, valA, valB)) }
                   .join(allCat) // Join with total number of reviews per category
                   .map { case (category, ((term, valA, valB), allReviews)) => 
                          ((category, term), valA, valB, allReviews - valA) } // Calculate C = allReviews - A

// ----> D value --> D = N - (A + B + C)
// D: number of reviews not in the current word's category without the current word
val ABCD = ABC.map { case ((category, word), valA, valB, valC) =>
  ((category, word), valA, valB, valC, N - (valA + valB + valC))
} // ((Category, Word), A, B, C, D)

//ABCD.take(10)

N: Long = 78829
A: org.apache.spark.rdd.RDD[((String, String), Int)] = ShuffledRDD[13] at reduceByKey at <console>:32
wordSum: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[15] at reduceByKey at <console>:33
AB: org.apache.spark.rdd.RDD[((String, String), Int, Int)] = MapPartitionsRDD[20] at map at <console>:36
allCat: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[22] at reduceByKey at <console>:42
ABC: org.apache.spark.rdd.RDD[((String, String), Int, Int, Int)] = MapPartitionsRDD[27] at map at <console>:46
ABCD: org.apache.spark.rdd.RDD[((String, String), Int, Int, Int, Long)] = MapPartitionsRDD[28] at map at <console>:51


In [5]:
// Calculate the chi squared values for all words
val chiSquared = ABCD.map { case ((category, word), valA, valB, valC, valD) =>
  val NF = N.toFloat
  val valAF = valA.toFloat
  val valBF = valB.toFloat
  val valCF = valC.toFloat
  val valDF = valD.toFloat
  ((category, word), NF * ((valAF * valDF - valBF * valCF) * (valAF * valDF - valBF * valCF)) /
   ((valAF + valBF) * (valAF + valCF) * (valBF + valDF) * (valCF + valDF)))
}

//chiSquared.take(10)

chiSquared: org.apache.spark.rdd.RDD[((String, String), Float)] = MapPartitionsRDD[29] at map at <console>:27


In [6]:
// Group chiSquared results based on 'Category'
val chiGrouped = chiSquared.map { case ((category, word), chiSq) => (category, (word, chiSq)) }
  .groupByKey()
  .mapValues(iter => iter.toList.sortBy { case (_, chiSq) => -chiSq }.take(75)) // Sort by chiSquared in descending order and take top 75

// Sort categories by name and format the output
val top75 = chiGrouped.sortByKey().map { case (category, words) =>
  val wordValuePairs = words.map { case (word, chiSq) => s"$word:$chiSq" }.mkString(" ")
  s"<$category> $wordValuePairs"
}

// Extract words from chiGrouped, remove values, ensure each word is unique, and sort them alphabetically
val mergedDict = chiGrouped.flatMap { case (category, words) =>
  words.map { case (word, chiSq) => word }}.distinct().sortBy(word => word)
// Save it to a single space separated line
val mergedString = mergedDict.collect().mkString(" ")
//mergedString.take(50)

chiGrouped: org.apache.spark.rdd.RDD[(String, List[(String, Float)])] = MapPartitionsRDD[32] at mapValues at <console>:28
top75: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[36] at map at <console>:31
mergedDict: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[45] at sortBy at <console>:38
mergedString: String = abq acdelco acne acoustic acre acting action actor actors actress acura adapter addario addicted addicting addictive ads ai aic aidan aimard aired airsoft albums alex almonds alpha alternator altima ameda amel amino ammo amp android animated animation anime answering antenna ants apos appetite apple applecare apply apps aquarium ar arai arch arrangements articulation artisan artist artists atdi atv audio author authors avent awesome babies babyface back backpacking ba...


In [7]:
// Collect top75 RDD to a list of strings and append
// the mergedDict string to the end, then convert it 
// back to RDD and save into a textfile
val top75List = top75.collect().toList
val combinedList = top75List :+ mergedString
val combinedRDD = sc.parallelize(combinedList).coalesce(1)
// save combinedRDD in a textfile
combinedRDD.saveAsTextFile("output.txt")

org.apache.hadoop.mapred.FileAlreadyExistsException:  Output directory hdfs://captain01.os.hpc.tuwien.ac.at:9000/user/e12347300/output.txt already exists

In [ ]:
// Get the saved file from HDSF
!hdfs dfs -get output.txt